In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from nflows.flows import MaskedAutoregressiveFlow

from counterfactuals.datasets import (
    AdultDataset,
    GermanCreditDataset,
    CompasDataset,
    HelocDataset,
    LawDataset,
    MoonsDataset,
)
from counterfactuals.discriminative_models import LogisticRegression, MultilayerPerceptron
from counterfactuals.optimizers.base import BaseCounterfactualModel
from counterfactuals.optimizers.ppcef import PPCEF

from counterfactuals.metrics.metrics import (
    evaluate_cf,
)
from sklearn.metrics import classification_report

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create dataset

In [4]:
# dataset = CompasDataset(file_path="../data/compas_two_years.csv")
dataset = AdultDataset(file_path="../data/adult.csv")
# dataset = GermanCreditDataset(file_path="../data/german_credit.csv")

In [6]:
disc_model = LogisticRegression(dataset.X_train.shape[1], 1)

# disc_model = MultilayerPerceptron([dataset.X_train.shape[1], 128, 1])
train_dataloader = dataset.train_dataloader(batch_size=128, shuffle=True, noise_lvl=0)
disc_model.fit(train_dataloader, epochs=20)
print(classification_report(dataset.y_test, disc_model.predict(dataset.X_test)))
disc_model.predict(dataset.X_test).shape

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 19, Loss: 0.4451: 100%|██████████| 20/20 [00:01<00:00, 15.86it/s]


              precision    recall  f1-score   support

         0.0       0.82      0.79      0.80       785
         1.0       0.80      0.83      0.81       784

    accuracy                           0.81      1569
   macro avg       0.81      0.81      0.81      1569
weighted avg       0.81      0.81      0.81      1569



(1569,)

# Relabeling

In [7]:
y_pred_train = disc_model.predict(dataset.X_train)
y_pred_test = disc_model.predict(dataset.X_test)
dataset.y_train = y_pred_train
dataset.y_test = y_pred_test

# noise_lvl - zaszumianie numerycznych cech treningowego datasetu
train_dataloader = dataset.train_dataloader(batch_size=128, shuffle=True, noise_lvl=1e-5)
test_dataloader = dataset.test_dataloader(batch_size=128, shuffle=False)

# Create flow model

In [8]:
# from nflows.flows import SimpleRealNVP

# flow = SimpleRealNVP(use_volume_preserving=True, features=dataset.X_train.shape[1], hidden_features=4, context_features=1, num_layers=5)

flow = MaskedAutoregressiveFlow(features=dataset.X_train.shape[1], hidden_features=4, num_blocks_per_layer=2, num_layers=1, context_features=1)

# Create cf class, train and test flow model

In [9]:
cf = PPCEF(gen_model=flow, disc_model=disc_model, disc_model_criterion=torch.nn.BCELoss(), checkpoint_path="model.pt", neptune_run=None)

In [11]:
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=20,
    patience=20,
    eps=1e-3, # eps for patience
)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 19, Train: -1.7291, test: -2.3006: 100%|██████████| 20/20 [00:03<00:00,  5.38it/s]


In [14]:
cf.test_model(test_loader=test_dataloader)

torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([33])


{'0.0': {'precision': 0.9315068493150684,
  'recall': 0.8994708994708994,
  'f1-score': 0.9152086137281292,
  'support': 756.0},
 '1.0': {'precision': 0.9094159713945172,
  'recall': 0.9384993849938499,
  'f1-score': 0.9237288135593219,
  'support': 813.0},
 'accuracy': 0.9196940726577438,
 'macro avg': {'precision': 0.9204614103547928,
  'recall': 0.9189851422323747,
  'f1-score': 0.9194687136437256,
  'support': 1569.0},
 'weighted avg': {'precision': 0.9200601420178038,
  'recall': 0.9196940726577438,
  'f1-score': 0.9196234782678103,
  'support': 1569.0}}

# Search counterfactuals

In [11]:
search_step_kwargs = {
    "alpha": 20,
    "beta": 0.1,
}
test_dataloader = dataset.test_dataloader(batch_size=16, shuffle=False)
Xs_cf, Xs_orig, ys_orig = cf.search_batch(
    dataloader=test_dataloader,
    epochs=1000,
    lr=0.005,
    **search_step_kwargs
)

  0%|          | 0/99 [00:00<?, ?it/s]

100%|██████████| 99/99 [01:58<00:00,  1.19s/it]


# Evaluate

In [12]:
evaluate_cf(
    cf_class=cf,
    disc_model=disc_model,
    X=Xs_orig,
    X_cf=Xs_cf,
    model_returned=np.ones(Xs_cf.shape[0]).astype(bool),
    continuous_features=dataset.numerical_features,
    categorical_features=dataset.categorical_features,
    X_train=dataset.X_train,
    y_train=dataset.y_train,
    X_test=dataset.X_test,
    y_test=dataset.y_test,
)

{'model_returned_smth': 1.0,
 'valid_cf_disc': 1.0,
 'dissimilarity_proximity_categorical_hamming': 1.0,
 'dissimilarity_proximity_categorical_jaccard': 1.0,
 'dissimilarity_proximity_continuous_manhatan': 0.5377690049382553,
 'dissimilarity_proximity_continuous_euclidean': 0.39735776260117645,
 'dissimilarity_proximity_continuous_mad': 7.009989652305443,
 'distance_l2_jaccard': 0.9553598342667539,
 'distance_mad_hamming': 1.445184418689292,
 'plausibility': 1.4445169624158827,
 'kde_log_density_cfs': -37.17960926968742,
 'kde_log_density_xs': 29.749573210099516,
 'sparsity': 1.0,
 'valid_cf_gen': 0.9120458891013384,
 'flow_log_density_cfs': -57.212868,
 'flow_log_density_xs': -39.244335}